# Linear Regression: Predicting Taxi-Out Delay

#### Task Details
1. Go throught the dataset and perform preprocessing and then perform a 90:10 split and for train and test pruposes.
2. Firstly label encode the columns which are required.
3. Your target or y variable is TAXI-OUT time. Use all 8 algorithms above on the dataset with loss score as RMSE (Root mean Square Error).
4. Now, One-Hot encode all the data points and preform the 3rd Step again.

Keep in mind that you will be using the same splitted dataframe for all the training and testing and should not split again.

##### Some models to consider:-

Linear Models:-

1. Linear Regression
2. Ridge Regression(Popularily L1)
3. Lasso Regression(Popularily L2)

Non linear Models:-

1. KNN model
2. SVR
3. Naive Bayes
4. Random Forest
5. LightGBM(Tree Based Model)

##### Expected Submission
Open Submission till end of July.
Submit a Notebook: End-Goal: We will see how label encoding or one hot encoding is better for the model and which out of the 8 algorithms which is the best. Just a cojparative study and plots to understand the results on bigger datasets.

##### Evaluation
Having a complete report along with comparative graphs.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

### Loading the data from csv file